# Supervised Learning

Complete all work in this notebook in the text and code cells provided, and as instructed. When you're ready to turn the notebook with your answers in for grading, Use the File menu to Download it as an `.ipynb` file. Submit that file as an attachment in Canvas.

Note: the output of the code you run in the notebook will not be saved with it, so make sure that your answers are captured in code cells or text cells.

## Background

This assignment is intended to introduce your to classical approaches to supervised learning -- specifically, the tasks of:
 + supervised _classification_ -- learning a classifier --  in which you have
  + a **training set**: a _training pattern matrix_ (an $n_{train}$ x _k_ pattern matrix with $n_{train}$ patterns, each containing _k_ features) accompanied by an $n_{train}$-element _training label vector_
  + a **testing set**: a $n_{test}$ x _k_ pattern matrix accompanied by an $n_{test}$-element _test label vector_
  
 + supervised _regression_ -- learning a function -- where you have data as above, but the training and testing labels are outputs of a function to be learned, rather than class labels. We're not doing regression in this assignment.

In either case, the task of learning (classifier or regressor) is followed by evaluation of the performance of the learned classifier/regressor on the testing data.

There are many ways to learn classifiers/regressors, and comparatively few ways to assess performance.

This assignment is focused on classification.

This assignment uses a lot of python Packages that may be new to you - you'll get a lot of hints, and you'll use only what you need to use. This software is used a lot in industry.

 + [**Pandas**](https://pandas.pydata.org) --- [Documentation](https://pandas.pydata.org/docs/)
 + [**SciKit-Learn**](https://scikit-learn.org/stable/) --- [User Guide](https://scikit-learn.org/stable/user_guide.html) --- [API documentation](https://scikit-learn.org/stable/modules/classes.html)
 + [**SciPy**](https://scipy.org) --- [Documentation](https://docs.scipy.org/doc/scipy/)
 + [**NumPy**](https://numpy.org) --- [Documentation](https://numpy.org/doc/stable)
 + [**Matplotlib**](https://matplotlib.org) --- [User Guide](https://matplotlib.org/stable/users/index) --- [Reference](https://matplotlib.org/stable/api/index.html)



In [ ]:
# import the things
import os
import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import gdown
import pandas as pd
import seaborn as sns

import sklearn.model_selection
import sklearn.preprocessing
import sklearn.neighbors
import sklearn.metrics
import sklearn.datasets

We'll start with some manual examination of a well known data set containing information about the passengers on _HMS Titanic_, famed in history and legend as the impossible-to-sink passenger liner that sank on her maiden voyage from England to the US in 1912.  This data set has served as Kaggle's introduction to suvervised learning competitions and the Kaggle page is [here](https://kaggle.com/c/titanic/overview).  We're using the complete data set rather than Kaggle's version, which is pre-divided into testing and training sets.

Our classification task is to predict passenger survival using four features: a socioeconomic status proxy measurement, age in years, the fare paid for passage, and a count of other relatives aboard. There are several features in the data set would not be expected to relate meaningfully to passenger survival prediction and those will be dropped.

First, get the data and do some basic data engineering. We're going to use  Pandas for this. You'll note it was already `imported` using the name `pd`. Using `pd` as an alias for `pandas` is a strong convention in the community.

In the code cell below, fetch the data. Luckily, one version of the Titanic data is available at [OpenML](https://openml.org), which means that we can use the `sklearn.datasets.fetch_openml()` function to get it.

 a. Use `fetch_openml()` to retrieve the data set into a variable called `titanic_bunch`. Its OpenML ID is 40945 (**note**: there are multiple versions of this data at OpenML; you _must_ use 40945). Add `parser='auto', as_frame=True, cache=true` as parameters to your call to `fetch_openml()` to suppress an annoying warning, return a sklearn `Bunch` object containing the data in a dataframe, and make a local copy, respectively.

 b. Extract the dataframe containing the data from `titanic_bunch`, and assign it to a variable named `raw_titanic_df`. To do this, treat the Bunch as if it were a dictionary. The dataframe is stored under the key named `frame`.

 c.  Use `display(raw_titanic_df)` to get a nice tabular view of the data set. To orient you: each row of the dataframe is a _pattern_ and each column is a _feature_.

 d. Use `display(raw_titanic_df.info())` to understand the memory footprint of the data set, the number of non-NULL entries (sparseness of the data), and the nyumer of features and their names.

In [ ]:
# A - Fetch data from OpenML
titanic_bunch = sklearn.datasets.fetch_openml(data_id=40945, parser='auto', as_frame=True, cache = True)

# B - Extract Dataframe
raw_titanic_df = titanic_bunch['frame']

# C - Get tabular view of dataframe
print("raw_titanic_df")
display(raw_titanic_df)
print('\n\n')

# D - Get memory footprint
print("Memory Footprint")
display(raw_titanic_df.info())


We need to do some _data engineering_ to get the data we want, and get rid of the data we don't want.  These sorts of manipulations are the things that Pandas is really good at.

We're going to do three things:
 + create a new feature `relatives` that is the sum of `sibsp` and `parch`. This will create a new column in `raw_titanic_df`. It seems (from the somewhat unclear documentation) that `sibsp` is "number of siblings present as passengers" and `parch` is "number of parents present as passengers", so the sum `relatives` will be the number of relatives present.
 + get rid of the features (columns) we don't need for the classifier design, creating a new dataframee in the process.
 + remove rows (patterns) from the new dataframe that have missing values, yielding another new dataframe.

The resulting dataframe will have six columns, one of which (`survived`) is the classification label we'll be trying to find, and the five others being the features we'll use later to learn and test a classifier that can predict `survived`.

a. Add `raw_titanic_df['parch']` and `raw_titanic_df['sibsp']` and assign the result to `raw_titanic_df['relatives']`. Pretty easy!

b. Use `raw_titanic_df`'s `.drop()` method to drop the columns named `name`, `sex`, `sibsp`, `parch`, `ticket`, `cabin`, `embarked`,`boat`, `body`, and `home.dest`.  Call the result `reduced_titanic_df`.

c. There are some patterns (rows) in `reduced_titanic_df` with `NaN` as the value in some coumns (features). `NaN` is an abbreviation for "not a number", and (in this context) means the data wasn't available because of some reason.  The two basic approaches for handling `NaN`s are (a) discard patterns containing them, and (b) invent ("impute") values for the missing features.
We're going with the first approach.

Use your `reduced_titanic_df`'s `.dropna()` method to drop the rows with `NaN`s present, naming the resulting data frame `unscaled_titanic_df`. `display()` the result. **In the text cell below the code cell, state how many rows were dropped by `dropna()`.**

In [ ]:
# A - Create the relatives field -> parch + sibsp
raw_titanic_df['relatives'] = raw_titanic_df['parch'] + raw_titanic_df['sibsp']
print('raw_titanic_df with relatives')
display(raw_titanic_df)
print('\n\n')

# B - Remove fields we dont need
reduced_titanic_df = raw_titanic_df.drop(columns = ['name', 'sex', 'sibsp', 'parch', 'ticket', 'cabin', 'embarked', 'boat', 'body', 'home.dest'] )
print('reduced_titanic_df')
display(reduced_titanic_df)
print('\n\n')

# C - Clean NaN values
unscaled_titanic_df = reduced_titanic_df.dropna()
print('unscaled_titanic_df')
display(unscaled_titanic_df)


It's time for a visualization interlude.

It's always nice to look at the data, and this is fairly doable for low-dimensional data like the Titanic data set (4 features is a **very** low dimensionality for AI/ML problems - note: the `survived` feature is the class label, so it's not a feature used to perform prediction; it's the _target_ of the prediction).  We do this to get a sense for the "layout" of the patterns and (for classification problems) the way the different labeled classes are distributed.  Appearances can be deceiving -- our eyes only see two dimensions,  so any decomposition of the true four-dimensional reality of the Titanic measurements, even one that considers all possible pairs, won't tell us everything about the layout of the penguins in that 4D space.

While Python has a very capable graphics library called [**Matplotlib**](https://matplotlib.org), there are other graphics libraries that use Matplotlib to do even cooler things. [**Seaborn**](https://seaborn.pydata.org)  is quite popular, and is `import`ed as `sns` above. The use of `sns` as an alias is another strong convention (ask the Internet why, and you'll uncover a connection to _The West Wing_ 🤓).

Use seaborn's `pairplot()` function to create a 4x4 _pair plot_ of the data in `unscaled_titanic_df`:
+ use the `survived` column as the `hue`.
+ use the other four columns as the `vars`.

Look at the pair plot and understand what the six subplots are depicting.
Our objective is to come up with a classifier that will accurately (perhaps even perfectly) separate the passengers into survivors or fatalities of the shipwreck. Answer these questions in the text cell below the code cell below.
 + Based on the appearance of the labeled points in the ten "off-axis" subplots, and referring to the legend to see the mapping between dot colors and survival, is there a single feature that separates the survivors from the fatalities?
 + List the pairs of feature names used in the plots that showed the best separation between the two classes.

In [ ]:
# Create 4x4 pair plot
sns.pairplot(data = unscaled_titanic_df, vars = ['pclass', 'age', 'fare', 'relatives'], hue='survived')

At this point, we have a dataframe `unscaled_titanic_df` with patterns and class labels. it's time to run experiments using it to design a classifier that will predict `survived` from `pclass`, `age`, `relatives`, and `fare`.  Since we have class labels, we can use them in the design of the classifier, and also use them to judge how well the classifier performs - this is so-called _supervised classification_ (labels are available).

All supervised classifiers use a _training set_ of data (patterns and associated labels) to _design_ the classifier, and a _testing set_ of data (patterns and associated "ground-truth" labels) to _evaluate_ it. The evaluation compares the classifier's output on the testing set ("predicted labels") to the "ground truth" labels for the same data in some way.

Unless you are in very special circumstances, the training and testing label sets must be disjoint (they must share no patterns and associated labels). If you "test on the training data", you will get an optimistically biased estimate of performance.

The function `sklearn.model_selection.train_test_split()` will perform this partitioning randomly. Since the `unscaled_titanic_df` contains both the patterns and the labels, you'll do the train/test split first, and then peel off the labels from each of the results.

 a. Use `train_test_split()` to partition `unscaled_titanic_df` into `temp_train` and `temp_test`. Do **not** specify a `random_state` - every student will work with slightly different partitions of the data. Specify `test_size` as 0.25, and specify `shuffle=True`.

 b. Now split `temp_train` into two dataframes named `temp_train_label` and `unscaled_train_pattern`. `temp_train_label` is the `survived` column of `temp_train` and `unscaled_train_pattern` is the other four columns of `temp_train`.  Hint: if you want to select some columns by name from a dataframe, supply a list (using square brackets) of the column names as an index (itself in square brackets), _e.g._:  `foo_df = bar_df[['baz','blurfl','bloop']]`.

 c. Using the same technique used in part c, split `temp_test` into the labels (named `temp_test_label`) and the patterns (`unscaled_test_pattern`).

 d. Use `display()` to display `temp_test_label` and `unscaled_test_pattern`.

 e. As a final step, obtain `train_label` from `temp_train_label` by converting it to a Numpy array and then raveling it: `train_label = temp_train_label.to_numpy().ravel()`. This allows you to avoid seeing a cryptic error message later.

 f. Using the method in (e), obtain `test_label` from `tempt_test_label` by convertiung to Numpy and raveling.

 (If you're curious about why we're using `unscaled_` on the pattern matrices, it's because we have more to do to them -- see below).

In [ ]:
# A - Partition the dataframe into training and testing data
temp_train, temp_test = sklearn.model_selection.train_test_split(unscaled_titanic_df, test_size = 0.25, shuffle = True)

print('temp_train')
display(temp_train)
print('\n\n')

print('temp_test')
display(temp_test)
print('\n\n')

# B - Split temp train
temp_train_label = temp_train['survived']
unscaled_train_pattern = temp_train[['pclass', 'age', 'fare', 'relatives']]

print('temp_train_label')
display(temp_train_label)
print('\n\n')

print('unscaled_train_pattern')
display(unscaled_train_pattern)

# C - Split temp test
temp_test_label = temp_test['survived']
unscaled_test_pattern = temp_test[['pclass', 'age', 'fare', 'relatives']]

print('temp_test_label')
display(temp_test_label)
print('\n\n')

print('unscaled_test_pattern')
display(unscaled_test_pattern)

# E - Convert temp_train_label to numpy array
train_label = temp_train_label.to_numpy().ravel()

print("train_label")
print(train_label)
print('\n\n')

# F - Convert temp_test_label to numpy array
test_label = temp_test_label.to_numpy().ravel()

print("test_label")
print(test_label)
print('\n\n')



We're going to be basing our classification strategy on distance -- new "test passengers" whose feature vectors [`age`, `fare`, `relatives`, `pclass`]  are "closer", in some 4-dimensional sense, to those of "training passengers" are likely to have shared their fate. In order for that idea to work, we need to have coordinates that are scaled appropriately to calculate distance. If you look at the feature values are not in the same units, nor do they have the same range of values. It's not possible to fix the units, but we can equalize the ranges - and that's what we will do. We're going to employ a scaler. sklearn has a few different ones, and we'll use the `StandardScaler`, which is as simple as they come. A `StandardScaler` can be used to transform a pattern matrix so that each column (feature) has a mean of zero and a variance of 1.0. The scaler can also be re-used (if you keep it around) to scale other pattern matrices using the same scale and offset factors that were learned from the first pattern matrix.

We're going to _learn_ a `StandardScaler` from the training data, then we'll _transform_ the training data using it, and then we'll _transform_ the testing data using it. You can't use testing data in the design of a Scaler - it's the same kind of prohibition as "don't test your classifier on training data".

a. Create an instance of a `sklearn.preprocessing.StandardScaler`. Call it `scaler`.

b. Use `scaler`'s `.fit()` method to _fit it to_ the training pattern matrix `unscaled_train_pattern`.

c. Use `scaler`'s `.transform()` method to _transform_ the training pattern matrix `unscaled_train_pattern`, yielding an output pattern matrix `train_pattern`.

d. Use `scaler`'s `.transform()` method to transform the test pattern matrix `unscaled_test_pattern`, yielding an output pattern matrix `test_pattern`.

e. compute and print out the $1\times 4$ sample mean vector and the $4 \times 4$ sample covariance matrix for both `test_pattern` and `train_pattern`. Use Numpy's `np.mean()` routine with `axis=0` to get the proper mean vector, and use Numpy's `np.cov()` routine with `rowvar=False` to get the proper covariance matrix.

Note: you will see the values change every time you run the notebook, because the code above that creates training and testing partitions performs its splits at random; so every time you run it, you'll have different testing and training data. Fun!

In [ ]:
# A - Declare and initialize the scaler
scaler = sklearn.preprocessing.StandardScaler()

# B - Fit scaler to training pattern
scaler.fit(unscaled_train_pattern)

# C - Transform train pattern using scaler
train_pattern = scaler.transform(unscaled_train_pattern)

print("train_pattern")
print(train_pattern)
print('\n\n')

# D - Transform test pattern using scaler
test_pattern = scaler.transform(unscaled_test_pattern)

print("test_pattern")
print(test_pattern)
print('\n\n')

# E - Mean Vector and Covariant Vector
mean = np.mean(test_pattern, axis = 0)
print('1x4 Mean of test_pattern')
print(mean)
print('\n\n')

covariant = np.cov(test_pattern, rowvar = False)
print('4x4 Covariance of test_pattern')
print(covariant)


Now we'll use one of the more intuitive approaches to classification to build a _passenger classifier_. The classifier's job is to take a 4-D pattern `[age, fare, relatives, pclass]`  and predict `survived`.

Our intuitive approach is the **_k_-nearest-neighbor classifier**.  Training this classifier is as simple as can be: just memorize the training patterns and their labels (store them somewhere). When a test pattern comes in to be classified, find the closest _k_ patterns, and return the class that represents the majority of those _k_ patterns.

Sklearn has a `sklearn.neighbors.KNeighborsClassifier` object to do this.

a. Create a `KNeighborsClassifier` object with `k=1`; name it `knnc`.  When calling the constructor, let the defaults apply for all other parameters.

b. Using `knnc`'s `.fit()` method, train it on the training patterns and labels.

c. Using `knnc`'s `.predict()` method, get predicted labels for the test patterns.

d. Use `sklearn.metrics.accuracy_score` to report the accuracy (fraction, expressed in percent, of samples classified correctly).

Note: for the same reasons as noted at the end of Problem 5, you may see your performance numbers change when you run the notebook again.

In [ ]:
# A - Creating neighbor classifier
knnc = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 1)

# B - Train neighbors on training data
knnc.fit(train_pattern, train_label)

# C - Predict labels for test patterns
predicted_label = knnc.predict(test_pattern)

# D - Get accuracy score
accuracy = sklearn.metrics.accuracy_score(test_label, predicted_label)
print(f'Accuracy: {accuracy*100:.2f}%')


The quantity _k_ in a _k_-nearest-neighbors classifier is an example of a _hyperparameter_ - a design quantity whose value is not immediately obvious, and thus should probably be explored to determine a quantity that yields optimal performance.

Let's explore some values of _k_.

a. Wrap the code from the prior cell in a _loop_ that iterates _k_ from 1 to 40. For each value of _k_, create and `.fit()` a _k_-NN classifier, use it to `.predict()` labels for the testing data, and compute the accuracy. Save the accuracies as you generate them.

b. Plot the accuracy of the _k_-NN classifier _versus_ k using a line plot. Set the range of the _y_-axis to [0.5,0.8] (don't let it autoscale the _y_-axis, in other words). However: if your plots are outside the range [0.5, 0.8], change the limits as needed to allow your graph to be completely visible. Add suitable labels to the x-axis, y-axis, and plot title.  You can do this in matplotlib (`plt.plot()`, `plt.ylim()`, `plt.xlabel()`, `plt.ylabel()`, `plt.title()`, and `plt.show()` might be useful here).

c. In the text cell below the code cell below, comment on the accuracy of your classifier. Does _k_ have a strong effect on the accuracy?

Note: for the same reasons as noted at the end of Problem 5, you may see your performance numbers change. But if you're seeing suspiciously equal accuracy for all values of _k_, make sure you don't have any bugs.

In [ ]:
# A - Iterate k from 1 to 40
accuracies = []
for k_value in range(1,41):
  knnc = sklearn.neighbors.KNeighborsClassifier(n_neighbors = k_value)

  knnc.fit(train_pattern, train_label)

  predicted_label = knnc.predict(test_pattern)

  accuracy = sklearn.metrics.accuracy_score(test_label, predicted_label)
  print(f'Accuracy k = {k_value}: {accuracy*100:.2f}%')
  accuracies.append(accuracy)

# B - Plot Accuracy vs K
print()
plt.plot(range(1,41), accuracies)
plt.title('Lineplot of Accuracy of the k-NN Classifier Versus k')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.ylim(0.5,0.8)
plt.show()

We'd really like to be able to explore multiple splits of the data into training and testing while we're iterating a classifier design. But we have to be careful. If we use the testing partition to evaluate the result of each design iteration, and choosing the best, we've let the testing data guide the design process.

The solution is to iterate through multiple splits of what we called the training data into two pieces: _training_ and _validation_ sets.  There's still a "final" test partition that we use to characterize expected performance. SciKit-learn's User Guide, [section 3.1](https://scikit-learn.org/stable/modules/cross_validation.html), is a reasonable guide to this concept, as well as a tour of the nitty-gritty details.

a. Using the code from problem 7 as a starting point, implement a five-fold cross-validation experiment that evaluates the performance of _k_-NN  classifiers five times for each value of _k_.  Aeach value of _k_, use `sklearn.model_selection.cross_val_score()`, supplying the _k_-NN classifier object for that value of _k_, the training patterns and labels, and a `cv` value of 5. Everything else can use the default values.  Save the accuracies as you generate them - you'll get five accuracies for each value of _k_ (not one, as in problem 7).

b.  Generate a plot of accuracy versus _k_, adding some razzle-dazzle:
 + compute the mean value of the five accuracies at each value of _k_, and plot that using a line plot. Make the color of that line plot green.
 + calculate both the maximum and minimum of the five accuracies for each value of _k_, and use `plt.fill_between()` to draw a shaded region between the min and max values (specify `alpha=0.2` to make it rather transparent).  That'll give you some idea of the variability in performance based on various selections of passengers to be in the training and testing sets.
 + You may adjust the bounds on the _y_-axis (using `plt.ylim()`) to allow the interesting part of the plot to fill the available range, with a bit of cushion (all of your plot including the shaded region must be visible).

c. In the text cell below the code cell below, address this question: did the use of five-fold cross-validation give you additional insight into performance of the _k_-NN classifier for this passenger fate classifier?

In [ ]:
# A - Five-fold Cross-validation Experiment
accuracies = []
for k_value in range(1,41):
  knnc = sklearn.neighbors.KNeighborsClassifier(n_neighbors = k_value)

  scores = sklearn.model_selection.cross_val_score(knnc, train_pattern, train_label, cv = 5)

  print(f'Mean Accuracy k = {k_value}: {scores.mean()*100:.2f}%')
  accuracies.append(scores)

# B - Plotting
means = [a.mean() for a in accuracies]
maxes = [max(a) for a in accuracies]
mins  = [min(a) for a in accuracies]

print()
plt.plot(range(1,41), means, 'g')
plt.title("Five-fold Cross-validation Experiment")
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.ylim(0.5,0.8)
plt.fill_between(range(1,41), maxes, mins, alpha = 0.2)
plt.show()